In [18]:
import pandas as pd
from mlxtend.frequent_patterns import fpgrowth, association_rules
from mlxtend.preprocessing import TransactionEncoder

data_folder = './data'
min_support = 0.0025

In [19]:
ratings = pd.read_csv(f'{data_folder}/ratings.csv')

ratings = ratings[ratings['rating'] > 3]

print(ratings.shape)
ratings.head()

(179039, 3)


,user_id,course_id,rating
0,1889878,CC0101EN,4.0
2,1990814,ML0120ENv3,4.5
3,380098,BD0211EN,5.0
6,367075,DS0301EN,4.5
8,600100,BD0211EN,4.0


In [ ]:
# transactions = ratings.groupby('user_id')['course_id'].apply(list).reset_index().rename(columns={'course_id': 'rated_courses'})
# transactions.to_pickle(f'{data_folder}/fp_growth_transactions.pkl')
transactions = pd.read_pickle(f'{data_folder}/fp_growth_transactions.pkl')

print(transactions.shape)
transactions.head()

(31944, 2)


,user_id,rated_courses
0,2,"[LB0105ENv1, PY0101EN, DA0101EN, BD0123EN, BD0..."
1,4,"[PY0101EN, BD0153EN, BD0101EN, BD0135EN, DS032..."
2,5,"[LB0107ENv1, ST0201EN, DB0151EN, DS0110EN, DS0..."
3,7,[BD0211EN]
4,8,"[BD0111EN, BD0212EN]"


In [21]:
min_support * transactions.shape[0]

79.86

In [22]:
rated_courses_list = transactions['rated_courses'].tolist()

# One-hot encode
te = TransactionEncoder()
te_ary = te.fit(rated_courses_list).transform(rated_courses_list)
encoded_transactions = pd.DataFrame(te_ary, columns=te.columns_)

print(encoded_transactions.shape)
encoded_transactions.head()

(31944, 136)


,AI0111EN,BC0101EN,BC0201EN,BC0202EN,BD0101EN,BD0111EN,BD0115EN,BD0121EN,BD0123EN,BD0131EN,...,WA0101EN,WA0103EN,excourse40,excourse46,excourse47,excourse52,excourse54,excourse69,excourse77,excourse84
0,False,True,False,False,True,True,False,False,True,True,...,True,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,True,True,True,True,True,...,False,True,False,False,False,False,False,False,False,False
2,False,True,False,False,False,False,False,False,True,False,...,False,True,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [ ]:
# frequent_itemsets = fpgrowth(encoded_transactions, min_support=min_support, use_colnames=True)
# frequent_itemsets.to_pickle(f'{data_folder}/fp_growth_frequent_itemsets.pkl')

frequent_itemsets = pd.read_pickle(f'{data_folder}/fp_growth_frequent_itemsets.pkl')

print(frequent_itemsets.shape)
frequent_itemsets.head()

(33490, 2)


,support,itemsets
0,0.359379,(PY0101EN)
1,0.317963,(BD0101EN)
2,0.255604,(BD0111EN)
3,0.200632,(DA0101EN)
4,0.185512,(DS0103EN)


In [26]:
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)
rules = rules[(rules['lift'] > 2) & (rules['leverage'] > 0.01)]

rules.to_pickle(f'{data_folder}/fp_growth_rules.pkl')

print(rules.shape)
rules.head()

(2146, 14)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
23,"(PY0101EN, BD0111EN, DS0101EN)",(BD0101EN),0.050056,0.317963,0.032494,0.649156,2.041610,1.0,0.016578,1.943989,0.537074,0.096846,0.485594,0.375676
26,"(PY0101EN, BD0111EN)","(BD0101EN, DS0101EN)",0.103525,0.143595,0.032494,0.313880,2.185867,1.0,0.017629,1.248185,0.605165,0.151400,0.198837,0.270086
27,"(PY0101EN, BD0101EN)","(BD0111EN, DS0101EN)",0.129445,0.103619,0.032494,0.251028,2.422608,1.0,0.019081,1.196815,0.674537,0.162010,0.164449,0.282311
29,"(BD0111EN, DS0101EN)","(PY0101EN, BD0101EN)",0.103619,0.129445,0.032494,0.313595,2.422608,1.0,0.019081,1.268282,0.655103,0.162010,0.211532,0.282311
30,"(BD0101EN, DS0101EN)","(PY0101EN, BD0111EN)",0.143595,0.103525,0.032494,0.226292,2.185867,1.0,0.017629,1.158673,0.633480,0.151400,0.136944,0.270086
